In [15]:

import json
import pandas as pd
import sqlite3
import numpy as np
import re

#### Evaluating the Full Dataset

The Yelp dataset contained a huge amount of information.  Overall, there were 5 .json files:

1. yelp_academic_dataset_business.json - Information on the businesses listed on Yelp including the business name, average rating, location, attributes (accepts credit card, outdoor seating, etc.), whether it was currently open or not, and its hours.  Contained 14 columns and about 150,000 rows.
2. yelp_academic_dataset_review.json - Information on individual reviews including the user id of the poster, the business id of the business reviewed, the number of stars, the text for the review, date posted, and how many users found the review either "useful", "funny", or "cool".  Contained 9 columns and about 7,000,000 rows.
3. yelp_academic_dataset_user.json - Information on individual Yelp accounts and profiles including name, number of total reviews, how long they had been using the platform, average star rating, elite status, friends and fans, number of "useful", "funny", and "cool" reviews, and various other interactins.  Contained 22 columns and about 2,000,000 rows.
4. yelp_academic_dataset_checkin.json - Simple information on check-ins - the date and the id of the business visited.  Contained 2 columns and about 132,000 rows.
5. yelp_academic_dataset_tip.json - Information on "tips".  "Tips" are shorter than full reviews and are generally 1-2 sentences providing relevant information for a given business.  This dataset had information on the user id of the poster, the business id, the text, the date posted, and how many compliments the tip received from other users.  Contained 5 columns and about 909,000 rows.

In [2]:
data_file_review = open('Data/yelp_academic_dataset_review.json', encoding = 'utf-8')
data_review = []
for line in data_file_review:
    data_review.append(json.loads(line))
review_df = pd.DataFrame(data_review)
data_file_review.close()

data_file_business = open('Data/yelp_academic_dataset_business.json', encoding = 'utf-8')
data_business = []
for line in data_file_business:
    data_business.append(json.loads(line))
business_df = pd.DataFrame(data_business)
data_file_business.close()

#### Creating a Relevant Dataset

Most of the relevant information needed for this project was contained in the review .json.  However, I needed to join it with the business .json because of the ~17,000,000 reviews, ~13,000,000 of them were for businesses that were not restaurants.  Because of the size of the dataset, I decided to focus only on restaurants for this project.

To perform this filtering, I used SQL to join on the business id and return the relevant information after filtering for non-restaurant businesses.  The resulting dataframe still contains columns that will not be used for this project, but I saved it as such to be used for future projects.  I saved this dataframe to a .csv file for use in this project, as running the notebook on the full dataset takes a very long time.  For this reason, I have isolated this process to a separate notebook and will be using the created dataset for the bulk of my work.

In [3]:
conn = sqlite3.connect('review.db')
business_df = business_df.astype(str)
review_df = review_df.astype(str)
review_df.to_sql(name='review', con=conn, if_exists='replace', index=False)
business_df.to_sql(name='business', con=conn, if_exists='replace', index=False)

In [4]:
query = """
SELECT review.review_id AS review_id,
        business.name AS business_name,
        business.stars AS business_rating,
        business.categories AS categories,
        review.stars AS stars,
        review.useful AS useful,
        review.funny AS funny,
        review.cool AS cool,
        review.text AS review_text
FROM review
        JOIN business
            ON review.business_id = business.business_id
WHERE INSTR(categories, 'Restaurant') > 0
"""

In [16]:
df = pd.DataFrame(pd.read_sql(query, conn))

In [19]:
df['stars'] = df['stars'].astype(np.float64)
pattern = r'[a-zA-Z0-9!@#$%^&*()-_=+{}[\]|;:\'",.<>?/\\]'
def contains_pattern(text):
    return bool(re.match(pattern, text))
mask = df['review_text'].apply(contains_pattern)
df = df[mask].drop(columns = ['review_id', 'business_name', 'business_rating', 'categories', 'useful', 'funny', 'cool'], axis = 1)
df = df[df['stars'] != 3]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4180948 entries, 0 to 4724683
Data columns (total 2 columns):
 #   Column       Dtype  
---  ------       -----  
 0   stars        float64
 1   review_text  object 
dtypes: float64(1), object(1)
memory usage: 95.7+ MB


In [20]:
df.isna().sum()

stars          0
review_text    0
dtype: int64

In [21]:
df.to_csv('yelp_restaurant_reviews.csv', index = False)